In [1]:
!pip install tflite-runtime
!pip install mediapipe

^C
^C


In [1]:
import tensorflow as tf
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os

In [2]:
FacesEmbedding=pd.read_csv("FinalEmbedding.csv",index_col=0)
persons=list(FacesEmbedding.columns)

In [3]:
FacesEmbedding.head()

,aboallil,kabel,mahmoud,shams
0,0.000000,0.234469,0.0,0.0
1,0.004311,0.000000,0.0,0.0
2,0.000000,0.000000,0.0,0.0
3,0.000000,0.000000,0.0,0.0
4,0.000000,0.000000,0.0,0.0


In [4]:
model_path="./FinalFaceModel30Epoce.tflite"
interpreter=tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()
input_details=interpreter.get_input_details()
output_detail=interpreter.get_output_details()

In [5]:
MediapipeModelPath="./mediapipemodels/face_landmarker.task"
BaseOptions=mp.tasks.BaseOptions
FaceLandMarker=mp.tasks.vision.FaceLandmarker
FaceLandMarkerOptions=mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode=mp.tasks.vision.RunningMode
FaceLandMarkerResult=mp.tasks.vision.FaceLandmarkerResult

In [6]:
options=FaceLandMarkerOptions(base_options=BaseOptions(model_asset_path=MediapipeModelPath),
                             running_mode=VisionRunningMode.IMAGE)
landmarker= FaceLandMarker.create_from_options(options)

In [25]:
StreamCapture=cv2.VideoCapture("./zz.mp4")

In [26]:
def render(results,FaceImage,persons):
    for res in results.face_landmarks:
        
        x_=int(res[145].x*frame.shape[1])
        y_=int(res[145].y*frame.shape[0])
        x2_=int(res[374].x*frame.shape[1])
        y2_=int(res[374].y*frame.shape[0])
        w=np.sqrt((x_-x2_)**2+(y_-y2_)**2)
        W=6.3
        f = 840
        d = (W * f) / w
        x=int(res[356].x*frame.shape[1])
        y=int(res[152].y*frame.shape[0])
        x2=int(res[162].x*frame.shape[1])
        y2=int(res[338].y*frame.shape[0])
        if x<FaceImage.shape[1]-10:
            x+=10
        if y>FaceImage.shape[0]-10:
            y+=10
        if x2>10:
            x2-=10
        if y2>10:
            y2-=10
        
        
        modelimg=mp_img.numpy_view()[y2:y,x2:x]
        FaceImage=cv2.rectangle(np.array(FaceImage),(x,y),(x2,y2),(255,0,0),3)
        cv2.putText(FaceImage,f"distance {d}",(x2,y), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),2)
        
        if modelimg.size<9:
            continue
        modelimg=cv2.resize(modelimg,(224,224)).astype(np.float32)
        modelimg=modelimg/255
        
        distances=[]
        if d>0:
            for index,name in enumerate(persons):
                interpreter.set_tensor(input_details[0]["index"],np.expand_dims(modelimg,axis=0))
                interpreter.invoke()
                output=interpreter.get_tensor(output_detail[0]["index"])[0]
                personimpeding=FacesEmbedding[name].values
                distance=np.sum(np.power(output-personimpeding,2))
                distances.append(distance)
            name=persons[np.argmin(distances)]
            distance=distances[np.argmin(distances)]
            if distance <0.29:
#                 modelimg=cv2.putText(modelimg,name+f"{distance}",(0,125), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),1)
                modelimg=cv2.putText(modelimg,name,(0,125), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),3)
                modelimg=cv2.cvtColor(modelimg,cv2.COLOR_RGB2BGR)            
                cv2.imshow(name,modelimg)
#                 FaceImage=cv2.putText(FaceImage,f"distance {distance}",(x2,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1)
                FaceImage=cv2.putText(FaceImage,f"name {name}",(x2,y2), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),3)
                FaceImage=cv2.cvtColor(FaceImage,cv2.COLOR_RGB2BGR)
                FaceImage=cv2.resize(FaceImage,dsize=(0,0),fx=0.5,fy=0.5)
                cv2.imshow("FaceImage",FaceImage)
                
                
            else:
#                 FaceImage=cv2.putText(FaceImage,f"distance {distance}",(x2,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255,255,255),1)
                FaceImage=cv2.putText(FaceImage,f"name UnKnow",(x2,y2), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),3)
                FaceImage=cv2.cvtColor(FaceImage,cv2.COLOR_RGB2BGR)
                FaceImage=cv2.resize(FaceImage,dsize=(0,0),fx=0.5,fy=0.5)
                cv2.imshow("FaceImage",FaceImage)
                

In [27]:

while StreamCapture.isOpened:

#     img=cv2.imread("./kl.jpg")
    _,frame=StreamCapture.read()
#     frame[2:img.shape[0]+2,5:img.shape[1]+5]=img
    frame=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    mp_img=mp.Image(image_format=mp.ImageFormat.SRGB,data=frame)
    result=landmarker.detect(mp_img)
    
    modelimg=render(result,mp_img.numpy_view(),persons)
    
    key=cv2.waitKey(1000)
    if key ==27:
        key=cv2.waitKey(0)
        cv2.destroyAllWindows()
        break

In [10]:
cv2.destroyAllWindows()